# Recommendation System

In [1]:
import pandas as pd
import numpy as np

## Задание 1

Объедини общие данные о фильмах [tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA) и каст фильмов
[tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9). Оставь в датасете только фильмы, которые вышли в "релиз".\
Выведи количество фильмов, оставшихся после фильтрации.

In [2]:
# # для запуска в jupyter notebooks:
df_movies = pd.read_csv('../datasets/tmdb_5000_movies.csv')
df_credits = pd.read_csv('../datasets/tmdb_5000_credits.csv')

# для запуска в goodle colab:
#df_movies = pd.read_csv('/content/tmdb_5000_movies.csv')
#df_credits = pd.read_csv('/content/tmdb_5000_credits.csv')

df = pd.merge(df_movies, df_credits, how='left', left_on='id', right_on='movie_id')

df = df[df['status'] == 'Released']

print(f'Количество фильмов, оставшихся после фильтрации: {len(df)}.')

Количество фильмов, оставшихся после фильтрации: 4795.


## Задание 2

Самый наивный подход к рекомендации фильмов - рекомендовать фильмы с лучшими оценками пользователей. Фильмы, которые пользуются большей популярностью и признанием критиков, с большей вероятностью понравятся среднему зрителю.

Для справедливой оценки фильмов возьмем текущую рейтинговую систему IMDB (weighted rating (WR)), которая рассчитывается по формуле:
$$WR = \frac{v}{v + m} ⋅ R + \frac{m}{v + m} ⋅ C$$
$v$ - количество голосов \
$m$ - количество голосов для включения в финальную таблицу \
$R$ - средняя оценка \
$C$ - средняя оценка всех фильмов

Имплементируй функцию `weighted_rating`. С её помощью расcчитай рейтинг для каждого фильма и сохрани его в колонку `simple_score`.\
Выведи топ-5 фильмов по получившемуся рейтингу.
> В качестве параметра $m$ выбери 95-й квантиль количества голосов.

In [3]:
def weighted_rating():
    v = df['vote_count']
    m = np.percentile(df['vote_count'], 95)
    R = df['vote_average']
    C = df['vote_average'].mean()
    return (v / (v + m)) * R + (m / (v + m)) * C

df['simple_score'] = weighted_rating()

top_5 = df.sort_values('simple_score', ascending=False)[:5]['title_x'].values.tolist()

print(f'Топ-5 фильмов по weighted rating (WR): \n{top_5}')
print(df.columns)

Топ-5 фильмов по weighted rating (WR): 
['The Shawshank Redemption', 'The Dark Knight', 'Fight Club', 'Inception', 'Pulp Fiction']
Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title_x', 'vote_average',
       'vote_count', 'movie_id', 'title_y', 'cast', 'crew', 'simple_score'],
      dtype='object')


## Задание 3

Такой подход к рекомендациям очень наивен, так как не учитывает информацию о самом фильме (жанр, режиссер, описание, актеры и т.п). \
**Content Based Filtering** (Фильтрация на основе содержания) - тип рекомендательной системы, которая предлагает пользователям похожие элементы на основе конкретного элемента. Общая идея этих рекомендательных систем заключается в том, что если человеку понравился определенный товар, то ему понравится и похожий на него товар.

<center><img src="../misc/images/content.png" alt= “” width="300" height="500">

Реализуем алгоритм рекомендации на основе описания фильма. Для это требуется провести предобработку текста:
* Замени NaN в описании фильма на пустой символ `''`
* Удали все английские стоп слова (используй параметр `stop_words` в `TfidfVectorizer`)
* Расcчитай [Tf-Idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) для описания фильма

Выведи размер получившейся матрицы Tf-Idf

> Для [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) используйте параметры по умолчанию

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vector = TfidfVectorizer(stop_words='english')
df['overview'] = df['overview'].fillna('')
matr = vector.fit_transform(df.overview)
print(f"Размер матрицы Tf-idf {matr.shape}")

Размер матрицы Tf-idf (4795, 20970)


## Задание 4

Теперь тебе необходимо вычислить показатель сходства между описаниями фильмов. Используем косинусное расстояние, оно рассчитывается по формуле:
$$cos(Θ) = \frac{A ⋅ B}{∥A∥ ∥B∥} = \frac{ Σ_{i=1}^{n} A_i ⋅ B_i } { \sqrt{Σ_{i=1}^{n}A_{i}^{2}} ⋅ {\sqrt{Σ_{i=1}^{n}B_{i}^{2}}}}$$
Но поскольку мы использовали векторизатор TF-IDF на предыдущем шаге, достаточно вычислить скалярное произведение, которое и даст оценку косинусного сходства. Рассчитать его можно через [linear_kernel](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.linear_kernel.html). Результат сохрани в переменную `cosine_sim`.

Выведи размер получившейся матрицы `cosine_sim`.

In [5]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(matr, matr)
print(f'Размер матрицы {cosine_sim.shape}')

Размер матрицы (4795, 4795)


## Задание 5

Напиши функцию `get_recommendations`. На вход она принимает:
* `movies_dataset` - датасет фильмов
* `title` - название фильма, для которого мы будем искать похожие
* `cosine_sim` - матрица расстояний между описаниями
* `top_k` - топ-k cхожих фильмов

Возвращает top_k названий фильмов, описание которых похоже на выбранный фильм.\
Выведи топ-5 фильмов для `title='Saving Private Ryan'`

In [6]:
def get_recommendations(movies_dataset, title, cosine_sim, top_k=10):

    # получаю Series с названиями фильмов и индексами:
    movie_index = pd.Series(movies_dataset.index, index = movies_dataset['title_x']).drop_duplicates()

    # получаю индекс фильма по его названию:
    index = movie_index[title]

    # получаю список похожих фильмов с фильмом по индексу:
    similar_movies = list(enumerate(cosine_sim[index]))

    # получаю отсортированный по убыванию список похожих фильмов без первого (самого себя):
    sorted_similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)[1:]

    # получаю индексы похожих фильмов согласно top_k:
    top_index = [score[0] for score in sorted_similar_movies[0: top_k]]

    # получаю топ названий фильмов по их индексам:
    top_k_movies = movies_dataset['title_x'].iloc[top_index].values.tolist()

    return top_k_movies

In [7]:
get_recommendations(movies_dataset=df, title='Saving Private Ryan', cosine_sim=cosine_sim)[:5]

['The Great Raid',
 'The Monuments Men',
 'The Expendables 2',
 'Abandoned',
 'The Train']

## Задание 6

Еще один подход к построению рекомендательной системы - подход на основе сходства между пользователями. Этот подход называется **Collaborative Filtering** (Коллаборативная фильтрация).
<center><img src="../misc/images/all.png" alt= “” width="600" height="700"></center>
Коллаборативная фильтрация - это тип рекомендательной системы, которая использует поведение и предпочтения похожих пользователей для рекомендации товаров или продуктов конкретному пользователю. Система собирает данные о прошлом поведении пользователей, такие как покупки, рейтинги и отзывы, и анализирует их для выявления закономерностей и сходства между пользователями. На основе этих закономерностей система рекомендует товары, которые понравились или были приобретены другими такими же пользователями в прошлом.

Для реализации Коллаборативной фильтрации нам потребуются оценки пользователей [ratings](../datasets/ratings.csv).

>userId - id пользователя \
movieId - id фильма \
rating - оценка фильма (от 0 до 5)\
timestamp - время оценки


Воспользуйся библиотекой [surprise](https://surpriselib.com/) для обучения модели оценки рейтинга фильма [SVD](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD). Выведи средние значения 'RMSE', 'MAE' на кросс-валидации с параметрами `cv=5`.

In [10]:
!pip install surprise

  Using cached scikit-surprise-1.1.3.tar.gz (771 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for scikit-surprise
Failed to build scikit-surprise
  Running setup.py install for scikit-surprise: started
  Running setup.py install for scikit-surprise: finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [76 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-310
  creating build\lib.win-amd64-cpython-310\surprise
  copying surprise\accuracy.py -> build\lib.win-amd64-cpython-310\surprise
  copying surprise\builtin_datasets.py -> build\lib.win-amd64-cpython-310\surprise
  copying surprise\dataset.py -> build\lib.win-amd64-cpython-310\surprise
  copying surprise\dump.py -> build\lib.win-amd64-cpython-310\surprise
  copying surprise\reader.py -> build\lib.win-amd64-cpython-310\surprise
  copying surprise\trainset.py -> build\lib.win-amd64-cpython-310\surprise
  copying surprise\utils.py -> build\lib.win-amd64-cpython-310\surprise
  copying surprise\__init__.py -> build\lib.win-amd64-cpython-310\surprise
  copying surprise\__main__.py -> build\lib.win-amd64-cpython-310\surprise
  creating build\lib

In [11]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

# формирую датасет из csv-файла:
#df_ratings = pd.read_csv('/content/ratings.csv')
df_ratings=pd.read_csv('../datasets/ratings.csv')

# создаю объект класса Reader для чтения датасета:
reader = Reader()

# читаю данные из датасета:
data = Dataset.load_from_df(df_ratings[['userId', 'movieId', 'rating']], reader)

# создаю алгоритм SVD согласно документации:
algo = SVD()

# запускаю 5-ти фолдовую кросс-валидацию:
result = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5)

print(f"Среднее значение 'RMSE': {result['test_rmse'].mean()}")
print(f"Среднее значение 'MAE': {result['test_mae'].mean()}")

ModuleNotFoundError: No module named 'surprise'